In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gym
import base
from src.agents import DeepQAgent
from src.util import JupyterCallback

SyntaxError: invalid syntax (deep_q_agent.py, line 305)

# Environment

In [ ]:
env = gym.make('SpaceInvaders-v0')

In [ ]:
env.observation_space

In [ ]:
env.action_space

# Agent

In [ ]:
callback = JupyterCallback()

In [ ]:
agent = DeepQAgent(env, learning_rate=0.001, frames_per_action=3)
agent

In [ ]:
# res_random = agent.play()
# res_random

In [ ]:
# res_random.mean()

In [ ]:
agent.observe(10000)

In [ ]:
agent.train(episodes=100, callback=JupyterCallback())

In [ ]:
# res_trained = agent.play()
# res_trained

In [ ]:
# res_trained.mean()